# PyTorch Assignment: Convolutional Neural Network (CNN)

Creating a CNN using the MNIST dataset.

In [1]:
%matplotlib inline

#importing libraries and modules
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torch.nn as nn
from torchvision import datasets, transforms
from tqdm.notebook import tqdm, trange
import numpy as np

#creating class cnn and defining layers
class MNIST_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size = 3, padding = 1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size = 3, padding = 1)
        self.conv3 = nn.Conv2d(64, 64, kernel_size = 3, padding = 1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size = 3, padding = 1)
        self.conn1 = nn.Linear(7*7*64, 256) #unsure about the dimensions. 
        self.conn2 = nn.Linear(256, 10)
        
    def forward(self, x):
        # Convolution 1
        x = self.conv1(x)
        x = F.relu(x)
        
        # Convolution 2
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size = 2)
        
        # Convolution 3
        x = self.conv3(x)
        x = F.relu(x)
        
        # Convolution 4
        x = self.conv4(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size = 2)
        
        # Fully Connected Layer 1
        x = x.view(-1, 7*7*64)
        x = self.conn1(x)
        x = F.relu(x)
        
        # Fully Connected Layer 2
        x = self.conn2(x)
        return x
    

#downloading train and test MNIST sets, converting to Tensors
#download parameter below downloads the data from the internet if it is not available at root
mnist_train = datasets.MNIST(root = './datasets', train = True, transform = transforms.ToTensor(), download = True)
mnist_test = datasets.MNIST(root = './datasets', train = False, transform = transforms.ToTensor(), download = True)

# DataLoader takes care of batching and shuffling
train_load = torch.utils.data.DataLoader(mnist_train, batch_size = 100, shuffle = True)
test_load = torch.utils.data.DataLoader(mnist_test, batch_size = 100, shuffle = False)

model = MNIST_CNN()
# print(model)

#loss and optimizer
cr_ent_loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# trange is short for tqdm(range(N))
for epoch in trange(3):
    for images, labels in tqdm(train_load):
        # zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        x = images
        y = model(x)
        loss = cr_ent_loss(y, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
## Testing
correct = 0
total = len(mnist_test)

with torch.no_grad():
    # Iterate through test set minibatchs 
    for images, labels in tqdm(test_load):
        # Forward pass
        x = images  # <---- change here 
        y = model(x)

        predictions = torch.argmax(y, dim=1)
        correct += torch.sum((predictions == labels).float())

print('Test accuracy: {}'.format(correct/total))
        


Test accuracy: 0.9887999892234802
